<a href="https://colab.research.google.com/github/BrendenLatham/Data_Processing_And_Visualization/blob/main/BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learning SQL with BigQuery
### The Austin bikeshare dataset

In [1]:
from google.colab import auth
auth.authenticate_user()
print('authenticated')

authenticated


Now that the all mighty google has approved me the fun can start

First I will look at the size of the data set we will be working with

In [ ]:
%%bigquery --project data-viz-338801

SELECT
  COUNT(*) as TotalRows
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips

,TotalRows
0,1424786


Now lets have a peek at the content

In [ ]:
%%bigquery --project data-viz-338801

SELECT *
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
LIMIT 5

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


Now I will look at the longest trip taken

In [ ]:
%%bigquery --project data-viz-338801

SELECT
  MAX(duration_minutes) as LongestTrip
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips

,LongestTrip
0,34238


Now let's see that is days

In [ ]:
print('trip in days', 34238/60/24)

trip in days 23.77638888888889


Next we'll do the average trip length

In [ ]:
%%bigquery --project data-viz-338801

SELECT
  AVG(duration_minutes) as AverageTrip
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips

,AverageTrip
0,30.870428


It looks like the average was just over half an hour

Now I will look at the average length of trips based on starting point

In [ ]:
%%bigquery --project data-viz-338801

SELECT start_station_name as StartingPoint, AVG(duration_minutes) as AveDurationMin
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
GROUP BY StartingPoint
ORDER BY AveDurationMin


,StartingPoint,AveDurationMin
0,Shop,6.800000
1,Mobile Station @ Unplugged,9.750000
2,Main Office,11.810289
3,Dean Keeton & Whitis,13.494125
4,22nd & Pearl,13.893964
...,...,...
188,Stolen,105.000000
189,cesar Chavez/Congress,114.000000
190,17th/Guadalupe,126.739316
191,6th/Lavaca,132.333710


It looks like the shortest average duration was from somewhere called "Shop" and the longest was from East 5th/Shady Ln

I saved this query as "t" so I could use it again to get the average of all the places.

In [ ]:
%%bigquery --project data-viz-338801

with t as (
SELECT AVG(duration_minutes) as AveDurationMin, start_station_name as StartingPoint
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
GROUP BY StartingPoint
ORDER BY AveDurationMin
)

SELECT AVG(AveDurationMin) as AveStartingPointAve
FROM t

,AveStartingPointAve
0,39.154658


It looks like the average of the mean of all these starting points is more than just the average trip length

This isn't a good representation of average trip time because by this metric a starting point used 1 time would hold the same weight as a starting point used 100 times. It was still a fun introduction to sub-querys though.

Let's follow along this line though and look at the trips made per starting point

In [ ]:
%%bigquery --project data-viz-338801

SELECT start_station_name as StartingPoint, COUNT(start_station_name) AS frequency
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
GROUP BY StartingPoint
ORDER BY frequency DESC

,StartingPoint,frequency
0,21st & Speedway @PCL,72799
1,Riverside @ S. Lamar,40635
2,City Hall / Lavaca & 2nd,36520
3,2nd & Congress,35307
4,Rainey St @ Cummings,34758
...,...,...
188,Marketing Event,4
189,Eeyore's 2018,2
190,Stolen,1
191,Eeyore's 2017,1


I would think that our most frequently used start stations would have the average station duration closest to the total average ride duration. Just for the sake of practice I'll see

I'll look at the top 5 most frequent starting points

In [ ]:
%%bigquery --project data-viz-338801

SELECT start_station_name AS StartingPoint, COUNT(start_station_name) AS frequency , AVG(duration_minutes) AS AveDurationMin 
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
GROUP BY StartingPoint
ORDER BY frequency DESC
LIMIT 5

,StartingPoint,frequency,AveDurationMin
0,21st & Speedway @PCL,72799,18.781604
1,Riverside @ S. Lamar,40635,36.835388
2,City Hall / Lavaca & 2nd,36520,28.411610
3,2nd & Congress,35307,30.442802
4,Rainey St @ Cummings,34758,36.316992


I could have also done this by putting a condition on a previously used query but this seemed neater.

I might have overdone it on todays assignment but I was bored and this was pretty interesting

Now for fun I will look at another way to get the second most frequent starting point, but I will do it by the starting point ID and without displaying the rest of the starting points. I will also use subquiries for practice

In [ ]:
%%bigquery --project data-viz-338801

with t as (
  SELECT start_station_id AS StartingId, COUNT(start_station_id) AS frequency
  FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
  GROUP BY StartingId
  ORDER BY frequency DESC
  LIMIT 2
)

SELECT StartingId, Frequency
FROM t
ORDER BY frequency ASC
LIMIT 1

,StartingId,Frequency
0,2575,49640


I could have done it by just displaying the first row in the table after removing the max from t, but I wanted to see if this would also work. This seems like it could be more convenient for getting the n'th row of a table.

The next problem will be to get how many trips lasted over an hour, how many trips were round trips, and how many trips were both.

First I will do the trips over an hour

In [ ]:
%%bigquery --project data-viz-338801

with t as (
SELECT trip_id AS Trip, duration_minutes as DurationMin
FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
)

SELECT COUNT(*) AS TripsOverHour
FROM t
WHERE DurationMin > 60

,TripsOverHour
0,123789


Now for round trips

In [ ]:
%%bigquery --project data-viz-338801

with t as (
  SELECT trip_id AS Trip, start_station_id AS StartId, end_station_id AS EndId
  FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
)

SELECT COUNT(*) AS RoundTrips
FROM t
WHERE cast(StartId as string) = cast(EndId as string)

,RoundTrips
0,223747


Notice I had to cast the columns I am comparing as strings because when pulled streight from the database they were not matching data types

Now to do trips that were both over 60 minutes and round trips

In [ ]:
%%bigquery --project data-viz-338801

with t as (
  SELECT trip_id AS Trip, start_station_id AS StartId, end_station_id AS EndId, duration_minutes AS DurationMin
  FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
)

SELECT COUNT(*) AS LongRoundTrips
FROM t
WHERE cast(StartId as string) = cast(EndId as string) and DurationMin > 60

,LongRoundTrips
0,53576


Next I will be doing some work with joins.

I wil join the rows of two tables to examine the property type and average trip duration of bike share stations

In [ ]:
%%bigquery --project data-viz-338801

with t as (
  SELECT *
  FROM bigquery-public-data.austin_bikeshare.bikeshare_stations
  INNER JOIN bigquery-public-data.austin_bikeshare.bikeshare_trips
  ON bikeshare_stations.name = bikeshare_trips.start_station_name
)

SELECT name AS Name, property_type AS PropertyType, AVG(duration_minutes) AS AveDurationMin
FROM t
GROUP BY name, property_type
ORDER BY PropertyType

,Name,PropertyType,AveDurationMin
0,Zilker Park West,None,27.168259
1,Toomey Rd @ South Lamar,None,28.115798
2,State Capitol @ 14th & Colorado,None,30.421842
3,Waller & 6th St.,None,23.265843
4,Pease Park,None,28.648889
...,...,...,...
91,21st & University,undetermined_parking,18.341927
92,East 4th & Chicon,undetermined_parking,31.429918
93,West & 6th St.,undetermined_parking,28.106093
94,East 6th & Pedernales St.,undetermined_parking,32.332959


Now to compare the average trip length for each property type. I shouldn't have to change much from the last query

In [2]:
%%bigquery --project data-viz-338801
with q as (
  with t as (
   SELECT *
   FROM bigquery-public-data.austin_bikeshare.bikeshare_stations
   INNER JOIN bigquery-public-data.austin_bikeshare.bikeshare_trips
   ON bikeshare_stations.name = bikeshare_trips.start_station_name
  )

  SELECT name AS Name, property_type AS PropertyType, AVG(duration_minutes) AS AveDurationMin
  FROM t
  GROUP BY name, property_type
  ORDER BY PropertyType
)
SELECT PropertyType, AVG(AveDurationMin) AS AveDurationMin
FROM q
GROUP BY PropertyType

,PropertyType,AveDurationMin
0,None,27.089831
1,undetermined_parking,27.066346
2,sidewalk,34.582067
3,nonmetered_parking,50.633040
4,paid_parking,30.478807
5,parkland,39.996877
6,,33.502832


It looks like the trips from parkland were longer than the trips from sidewalk

For this next part I will answer a question from a classmate. The question I will answer is what council district in austin had the most, and longest average bikerides, and how does this correlate with the location of schools in the area.

In [40]:
%%bigquery --project data-viz-338801

with q as (
  with t as (
    SELECT *
    FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
    LEFT JOIN bigquery-public-data.austin_bikeshare.bikeshare_stations
    ON bikeshare_trips.start_station_id = bikeshare_stations.station_id
  )
  SELECT trip_id AS Trip, station_id AS Station, council_district AS District, subscriber_type AS Type, start_time AS Date, duration_minutes AS DurationMin
  FROM t
)
SELECT District, COUNT(District) AS Frequency, AVG(DurationMin) AS AveDurationMin
FROM q
GROUP BY District
Order BY Frequency DESC

,District,Frequency,AveDurationMin
0,9.0,1056038,28.999327
1,3.0,100457,36.137452
2,1.0,100315,32.689119
3,5.0,88340,37.980269
4,8.0,51006,42.587637
5,10.0,4855,47.403913
6,NaN,0,29.120715


It looks like district 9 had the most rides, while district 10 (the least frequent), had the longest average ride duration. It seems as though the less frequent a district is, the longer the average trip is in that district. This could be caused by many factors. It could be interpreted as the larger the frequency the more likely the average trip duration in that district is to match the total average duration, which recall was 30 minutes. However if this was a case of this common behavior in statistics we would expect to see some average durations the same distance lower than the mean as there are higher. In this case they are all higher.

This behavior could also be caused by the structure of the districts themselves. The districts of lower frequency most likely have a lower population, resulting in a more sparse layout of destinations, requiring longer trips. 

My favorite possibility though, and in my opinion the most likely, is that district 9 lies in a hub of the city, while the lower frequencies (and higher average ride times) correspond to the distance of the districts from the hub. This would make sense in common city layouts.

I think the second part of the question might help us out. In the subscriber type category there is a high frequency type for "U.T. Student Membership". If we see most of these in disctrict 9 it would make the hub theory just slightly more valid, as college areas are typically hubs.

In [71]:
%%bigquery --project data-viz-338801

with w as (
  with q as (
    with t as (
      SELECT *
      FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
      LEFT JOIN bigquery-public-data.austin_bikeshare.bikeshare_stations
      ON bikeshare_trips.start_station_id = bikeshare_stations.station_id
    )
    SELECT trip_id AS Trip, station_id AS Station, council_district AS District, subscriber_type AS Type, start_time AS Date, duration_minutes AS DurationMin
    FROM t
  )
  SELECT District, Type
  FROM q
  WHERE Type = "U.T. Student Membership"
)
SELECT District, COUNT(District) AS UTStudents
FROM w 
GROUP BY District
ORDER BY UTStudents DESC

,District,UTStudents
0,9.0,269941
1,1.0,10863
2,3.0,1707
3,5.0,1664
4,8.0,844
5,10.0,560
6,NaN,0


It looks like we have our answer. This isn't exactly conclusive but it is a pretty reasonalbe theory.

Okay you caught me I was the one who asked this question. I did ask a smaller question in class, but from that this question appeared and I wanted to experiment a bit.